In [1]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [2]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/arenda-pomescheniy?per-page=50'

Добыча колличества ссылок

In [3]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

735


Составляем массив ссылок

In [4]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 16/16 [00:07<00:00,  2.24it/s]

735


Составление своей базы данных

In [5]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    try:
        w2 = soup.find('table', id = 'w2')
        if w2:
            w2 = w2.find_all('tr')
            for tr in w2:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('w2 not found')

    try:
        properties = soup.find('div', class_ = 'properties m-t-2')
        if properties:
            properties = properties.find_all('div', class_ = 'property')
            for property in properties:
                key = property.find('div', class_ = 'property-name').text.strip()
                val = property.find('div', class_ = 'property-value').text.strip()
                if key not in tmp.keys(): # записи в 2х таблицах могут 
                    tmp[key] = val # пересекаться, а зачем нам копии?
    except:
        print('properties m-t-2 not found')
    
    try:
        buildings = soup.find('div', 'around').find_all('div', 'group')
        for building in buildings:
            name = building.find('div','group-body').find('span').text
            dist = building.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            tmp[name] = dist   
    except:
        pass

    df = pd.concat([df, pd.DataFrame(tmp)], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда помещений'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 735/735 [03:53<00:00,  3.15it/s]


Посмотрим на результаты вкрадце

In [6]:
df.head()

,Ссылка,Заголовок,Цена,Описание,№,Район,Адрес,Площадь,Год постройки,Этаж / этажность,...,Торг уместен,Телефон,Интернет,Мебель,Этажность,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/arenda-pomescheniy/ul-p...,Аренда помещения на ул. Порт-АртурскаяЛенински...,720 000 руб./мес.,На охраняемой территории предприятия сдается х...,2831160,Ленинский,ул. Порт-Артурская,4 000 м²,1980,1 / 1,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда помещений,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/arenda-pomescheniy/ul-m...,"Аренда помещения на ул. Максима Горького, д. 2...",103 480 руб./мес.,Красная линия.\n\nПервый этаж жилого дома «ЖК ...,2826909,Нижегородский,"ул. Максима Горького, д. 24","79,6 м²",NaN,1 / 12,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда помещений,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/arenda-pomescheniy/ul-u...,"Аренда помещения на ул. Ульянова, д. 12Нижегор...",170 400 руб./мес.,Сдается коммерческое помещение - 284 кв.м. Сам...,2809347,Нижегородский,"ул. Ульянова, д. 12",284 м²,NaN,1 / 5,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда помещений,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/arenda-pomescheniy/ul-z...,"Аренда помещения на ул. Заломова, д. 13БНижего...",37 700 руб./мес.,"Офис расположен в Нижегородском районе, на ули...",2838642,Нижегородский,"ул. Заломова, д. 13Б",58 м²,NaN,1 / 1,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда помещений,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/arenda-pomescheniy/ul-k...,"Аренда помещения на ул. Культуры, д. 11 к1Сорм...",80 000 руб./мес.,Сдаётся торговое помещение с отдельным входом ...,2820851,Сормовский,"ул. Культуры, д. 11 к1",70 м²,NaN,1 / 9,...,NaN,NaN,NaN,NaN,NaN,gipernn,Комерческая недвижимость,Аренда помещений,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [7]:
df.to_excel('indoor-rent.xlsx', index=False)